## 03. Modélisation 

L’objectif de cette section est d’évaluer l’impact du rééquilibrage par **SMOTE** (appliqué uniquement sur le jeu d’entraînement) sur les performances de modèles de classification. Nous analyserons la capacité du modèle à détecter la classe minoritaire avant et après rééquilibrage. 

Nous comparons deux modèles : **régression logistique** et **Random Forest**, appliquée sans rééquilibrage de la base de données et avec réequilibrage.

Dans un contexte de classes déséquilibrées, nous analysons le ROC-AUC et la matrice de confusion de chaque modèle afin d’étudier le compromis entre :
- FN : résiliations non détectées (coût potentiellement élevé),
- FP : fausses alertes (coût commercial / opérationnel).

/!\ Certaines parties de ce notebook prennent plusieurs minutes à être exécutées.

### 1- Importation des données et packages 

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report, roc_auc_score, confusion_matrix
)
from sklearn.model_selection import RandomizedSearchCV

import statsmodels.api as sm
from imblearn.over_sampling import SMOTE

In [ ]:
# Important : la séparation train/test a été faite en amont (notebook 2).
IN_DIR = Path("..") / "data" / "processed"

data = np.load(IN_DIR / "eudirectlapse_num_scaled_split.npz", allow_pickle=True)

X_train_num_scaled = data["X_train"]
X_test_num_scaled = data["X_test"]
y_train = data["y_train"]
y_test = data["y_test"]
num_cols = data["num_cols"].tolist()

scaler = joblib.load(IN_DIR / "scaler_num.joblib")

print(X_train_num_scaled.shape, X_test_num_scaled.shape)

(17295, 12) (5765, 12)


In [75]:
print("Train target rate:", y_train.mean())
print("Test target rate:", y_test.mean())

Train target rate: 0.1281295172015033
Test target rate: 0.1280138768430182


### 2- Régression logistique 
La régression logistique est un modèle linéaire simple et interprétable.  
Dans un contexte de déséquilibre, elle peut privilégier la classe majoritaire, ce qui se traduit par un faible nombre de résiliations détectées.

#### a - Modèle de référence : Régression logistique (sans SMOTE)

Une régression logistique est estimée à l’aide de statsmodels afin d’obtenir :
- les coefficients du modèle,
- les erreurs standards,
- les p-values associées à chaque variable explicative.

Cette approche permet d’analyser l’impact individuel de chaque variable sur la probabilité de résiliation et d’identifier les variables statistiquement non significatives.


In [ ]:
X_train_df = pd.DataFrame(X_train_num_scaled, columns=num_cols)
X_test_df  = pd.DataFrame(X_test_num_scaled,  columns=num_cols)
X_train_sm = sm.add_constant(X_train_df, has_constant="add")

logit_full = sm.Logit(y_train, X_train_sm)
res_full = logit_full.fit(disp=False, method="lbfgs", maxiter=200)

print(res_full.summary())

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                17295
Model:                          Logit   Df Residuals:                    17283
Method:                           MLE   Df Model:                           11
Date:                Sat, 03 Jan 2026   Pseudo R-squ.:                 0.01676
Time:                        15:00:45   Log-Likelihood:                -6509.8
converged:                       True   LL-Null:                       -6620.8
Covariance Type:            nonrobust   LLR p-value:                 2.041e-41
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -1.9619      0.024    -82.868      0.000      -2.008      -1.916
polholder_age         -0.1403      0.028     -4.978      0.000      -0.196      -0.085
policy_age          

Plusieurs variables liées aux montants de prime présentent des p-values égales à 1 (prem_final, prem_last,prem_charge_rel). De plus, la variable policy_nbcontract a un p-value de 0,66 donc la variable n'a aucun effet significatif. Les variables prem_market et prem_vs_market ne sont pas significatives.

Seules les variables de variation relative de prime ont été conservées.

In [96]:
vars_to_drop = [
    "prem_last",
    "prem_final",
    "prem_change_abs",
    "prem_vs_market",
    "prem_market",
    "policy_nbcontract"
]

X_train_reduced = X_train_df.drop(columns=vars_to_drop)
X_test_reduced  = X_test_df.drop(columns=vars_to_drop)

X_train_sm = sm.add_constant(X_train_reduced, has_constant="add")

res_clean = sm.Logit(y_train, X_train_sm).fit(disp=False, method="lbfgs", maxiter=200)
print(res_clean.summary())

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                17295
Model:                          Logit   Df Residuals:                    17288
Method:                           MLE   Df Model:                            6
Date:                Sat, 03 Jan 2026   Pseudo R-squ.:                 0.01263
Time:                        15:24:29   Log-Likelihood:                -6537.2
converged:                       True   LL-Null:                       -6620.8
Covariance Type:            nonrobust   LLR p-value:                 1.701e-33
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -1.9515      0.023    -83.174      0.000      -1.997      -1.906
polholder_age         -0.1565      0.028     -5.586      0.000      -0.211      -0.102
policy_age          

Une fois la sélection des variables effectuées nous pouvons entrainer le modèle avec sklearn.

In [97]:
model1 = LogisticRegression(max_iter=1000)
model1.fit(X_train_reduced, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

Le paramètre max_iter est fixé à 1000 afin de garantir la convergence de l’algorithme d’optimisation, en particulier dans un contexte de données déséquilibrées et standardisées.

In [98]:
y_pred_base = model1.predict(X_test_reduced)
y_proba_base = model1.predict_proba(X_test_reduced)[:, 1]

In [106]:
roc_auc = roc_auc_score(y_test, y_proba_base)

# Tableau récapitulatif
metrics_summary = pd.DataFrame({
    "Metric": ["ROC-AUC"],
    "Value": [roc_auc]
})

print("=== Performance summary ===")
display(metrics_summary)

print("=== Confusion matrix ===")
display(pd.DataFrame(
    confusion_matrix(y_test, y_pred_base),
    index=["True 0", "True 1"],
    columns=["Pred 0", "Pred 1"]
))

=== Performance summary ===


,Metric,Value
0,ROC-AUC,0.571412


=== Confusion matrix ===


,Pred 0,Pred 1
True 0,5027,0
True 1,738,0


Avec le seuil de décision par défaut (0.5), le modèle ne prédit aucune résiliation. Ce comportement s’explique par le fort déséquilibre de la variable cible. Bien que l’AUC indique une capacité de discrimination supérieure au hasard (0,57 > 0,5), le modèle ne peut pas être utilisé pour étudier les résiliations. Plusieurs approches peuvent alors être envisagées pour améliorer la détection de la classe minoritaire :

– un ajustement du seuil de décision 

– un rééquilibrage des classes avec la méthode SMOTE.

#### b - Régression logistique avec rééquilibrage (SMOTE)

Dans cette section, nous évaluons l’impact du rééquilibrage des classes par la méthode SMOTE sur les performances d’une régression logistique.  
SMOTE est appliqué uniquement sur le jeu d’entraînement (après sélection des variables) afin d’éviter toute fuite d’information vers le jeu de test.


In [101]:
smote = SMOTE(random_state=42)

X_train_smote, y_train_smote = smote.fit_resample(
    X_train_reduced ,
    y_train
)

print("Distribution après SMOTE :")
print(pd.Series(y_train_smote).value_counts(normalize=True))

Distribution après SMOTE :
0    0.5
1    0.5
Name: proportion, dtype: float64


Après application de SMOTE, la distribution de la variable *lapse* dans le jeu
d’entraînement est équilibrée. Les observations synthétiques sont générées
uniquement à partir des données d’apprentissage. On peut à nouveau effectuer une régression logistique sur cette nouvelles base des données. 


In [102]:
model_smote = LogisticRegression(max_iter=1000)
model_smote.fit(X_train_smote, y_train_smote)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [104]:
y_pred_smote = model_smote.predict(X_test_reduced)
y_proba_smote = model_smote.predict_proba(X_test_reduced)[:, 1]

In [ ]:
roc_auc = roc_auc_score(y_test, y_proba_smote)

metrics_smote = pd.DataFrame({
    "Metric": ["ROC-AUC"],
    "Value": [roc_auc]
})

print("=== Logistic Regression + SMOTE — Performance summary ===")
display(metrics_smote)

print("=== Confusion matrix ===")
display(pd.DataFrame(
    confusion_matrix(y_test, y_pred_smote),
    index=["True 0", "True 1"],
    columns=["Pred 0", "Pred 1"]
))

print("=== Classification report ===")
print(classification_report(y_test, y_pred_smote, digits=4))

=== Logistic Regression + SMOTE — Performance summary ===


,Metric,Value
0,ROC-AUC,0.570574


=== Confusion matrix ===


,Pred 0,Pred 1
True 0,2619,2408
True 1,311,427


=== Classification report ===
              precision    recall  f1-score   support

           0     0.8939    0.5210    0.6583      5027
           1     0.1506    0.5786    0.2390       738

    accuracy                         0.5284      5765
   macro avg     0.5222    0.5498    0.4487      5765
weighted avg     0.7987    0.5284    0.6046      5765



L’application de SMOTE modifie le comportement de la régression logistique. Le ROC-AUC reste du même ordre de grandeur que pour le modèle sans SMOTE. L’analyse des métriques usuels sur la classe minoritaire met en évidence une amélioration de la détection des résiliations.

La matrice de confusion montre que le modèle après rééquilibrage identifie correctement 427 résiliations (TP), contre aucune pour le modèle sans SMOTE. Le rappel de la classe minoritaire atteint ainsi 57,8 %, alors le modèle détecte correctement 57,8 % des résiliations. On remarque aussi que cette amélioration s’accompagne d’une augmentation du nombre de faux positifs (2 408).

Ces résultats confirment que SMOTE permet de rendre le modèle sensible à la classe d’intérêt, au prix d’un compromis entre détection des résiliations et augmentation des fausses alertes.

### 3 - Random Forest 
Dans cette section, nous évaluons les performances d’un modèle de Random Forest, d’abord sans rééquilibrage des classes, puis avec application de SMOTE sur le jeu d’entraînement.

La Random Forest est un modèle non-linéaire basé sur des arbres de décison. Elle est souvent plus flexible, mais peut également être affectée par le déséquilibre des classes.  

Contrairement à la régression logistique, la Random Forest ne repose pas sur des hypothèses de linéarité ou d’indépendance entre les variables. L’ensemble des variables explicatives est donc conservé.

#### a- Modèle de référence : Random Forest sans SMOTE



Un modèle de Random Forest de référence est tout d’abord entraîné à l’aide d’hyperparamètres standards afin d’établir un point de comparaison. Cette étape permet d’évaluer l’apport réel de l’optimisation des hyperparamètres réalisée par la suite.

In [112]:
rf_base = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

rf_base.fit(X_train_num_scaled, y_train)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",200
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [ ]:
y_pred_rf_base = rf_base.predict(X_test_num_scaled)
y_proba_rf_base = rf_base.predict_proba(X_test_num_scaled)[:, 1]

In [ ]:
roc_auc_rf_base = roc_auc_score(y_test, y_proba_rf_base)
print(f"ROC-AUC Random Forest : {roc_auc_rf_base:.4f}")

cm_rf_base = confusion_matrix(y_test, y_pred_rf_base)

display(pd.DataFrame(
    cm_rf_base,
    index=["True 0", "True 1"],
    columns=["Pred 0", "Pred 1"]
))

ROC-AUC Random Forest (baseline) : 0.5494


,Pred 0,Pred 1
True 0,5008,19
True 1,734,4


##### Optimisation des hyperparamètres

La performance d’une Random Forest dépend fortement de plusieurs hyperparamètres :  la profondeur des arbres et le nombre minimal d’observations par feuille. Une recherche des hyperparamètres est réalisée afin d’identifier une configuration optimale tout en limitant le coût de calcul.

L’optimisation est conduite à l’aide d’une validation croisée à cinq folds.

In [115]:
param_dist = {
    "n_estimators": [200, 300, 500],
    "max_depth": [None, 5, 10],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5]
}

rf = RandomForestClassifier(
    random_state=42,
    n_jobs=-1
)

rs = RandomizedSearchCV(
    rf,
    param_distributions=param_dist,
    n_iter=10,
    scoring="roc_auc",
    cv=5,
    verbose=1,
    random_state=42
)

rs.fit(X_train_num_scaled, y_train)

print("Meilleurs hyperparamètres :")
print(rs.best_params_)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Meilleurs hyperparamètres :
{'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 5}


In [130]:
rf_best = rs.best_estimator_

y_proba_rf_best = rf_best.predict_proba(X_test_num_scaled)[:, 1]
y_pred_rf_best = rf_base.predict(X_test_num_scaled)

In [132]:
roc_auc = roc_auc_score(y_test, y_proba_rf_best)

metrics_rf_base = pd.DataFrame({
    "Metric": ["ROC-AUC"],
    "Value": [roc_auc]
})

print("=== Random Forest — Performance summary ===")
display(metrics_rf_base)

print("=== Confusion matrix ===")
display(pd.DataFrame(
    confusion_matrix(y_test, y_pred_rf_best),
    index=["True 0", "True 1"],
    columns=["Pred 0", "Pred 1"]
))

print("=== Classification report ===")
print(classification_report(y_test, y_pred_rf_best, digits=4))


=== Random Forest — Performance summary ===


,Metric,Value
0,ROC-AUC,0.587906


=== Confusion matrix ===


,Pred 0,Pred 1
True 0,5008,19
True 1,734,4


=== Classification report ===
              precision    recall  f1-score   support

           0     0.8722    0.9962    0.9301      5027
           1     0.1739    0.0054    0.0105       738

    accuracy                         0.8694      5765
   macro avg     0.5230    0.5008    0.4703      5765
weighted avg     0.7828    0.8694    0.8124      5765



Comme la régression logistique, l’algorithme de random forest entraîné sans rééquilibrage a tendance à privilégier la classe majoritaire et peine à détecter les résiliations même après optimisation des paramètres.

#### b- Random Forest avec SMOTE

On applique SMOTE aux données d’entraînement (variables numériques standardisées), puis on entraîne la Random Forest sur ce jeu de données rééquilibré. 

In [124]:
smote = SMOTE(random_state=42)
X_train_rf_smote, y_train_rf_smote = smote.fit_resample(X_train_num_scaled, y_train)

rf_smote = RandomForestClassifier(
    n_estimators=300,
    max_depth=5,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)

rf_smote.fit(X_train_rf_smote, y_train_rf_smote)

y_pred_rf_smote = rf_smote.predict(X_test_num_scaled)
y_proba_rf_smote = rf_smote.predict_proba(X_test_num_scaled)[:, 1]

In [125]:
roc_auc = roc_auc_score(y_test, y_proba_rf_smote)

metrics_rf_smote = pd.DataFrame({
    "Metric": ["ROC-AUC"],
    "Value": [roc_auc]
})

print("=== Random Forest + SMOTE — Performance summary ===")
display(metrics_rf_smote)

print("=== Confusion matrix ===")
display(pd.DataFrame(
    confusion_matrix(y_test, y_pred_rf_smote),
    index=["True 0", "True 1"],
    columns=["Pred 0", "Pred 1"]
))

print("=== Classification report ===")
print(classification_report(y_test, y_pred_rf_smote, digits=4))

=== Random Forest + SMOTE — Performance summary ===


,Metric,Value
0,ROC-AUC,0.579823


=== Confusion matrix ===


,Pred 0,Pred 1
True 0,3892,1135
True 1,492,246


=== Classification report ===
              precision    recall  f1-score   support

           0     0.8878    0.7742    0.8271      5027
           1     0.1781    0.3333    0.2322       738

    accuracy                         0.7178      5765
   macro avg     0.5330    0.5538    0.5297      5765
weighted avg     0.7969    0.7178    0.7510      5765



L’application de SMOTE à la base d'apprentissage améliore la capacité de la Random Forest à détecter la classe minoritaire, comme en témoigne l’augmentation du nombre de résiliations correctement identifiées (TP). Toutefois, le gain reste plus limité que pour la régression logistique, et s’accompagne d’une augmentation du nombre de faux positifs.

Ces résultats suggèrent que, dans ce jeu de données, la Random Forest bénéficie moins du rééquilibrage par SMOTE que la régression logistique.


Une alternative au sur-échantillonnage consiste à utiliser la pondération des classes via le paramètre `class_weight="balanced"`.

Cette méthode pénalise davantage les erreurs commises sur la classe minoritaire, sans générer de données synthétiques.

Elle est souvent plus stable et plus simple à déployer en production.

In [128]:
rf_weighted = RandomForestClassifier(
    n_estimators=300,
    max_depth=5,
    min_samples_leaf=2,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)

rf_weighted.fit(X_train_num_scaled, y_train)

y_pred_rf_weighted = rf_weighted.predict(X_test_num_scaled)
y_proba_rf_weighted = rf_weighted.predict_proba(X_test_num_scaled)[:, 1]


In [129]:
roc_auc = roc_auc_score(y_test, y_proba_rf_weighted)

metrics_rf_weighted = pd.DataFrame({
    "Metric": ["ROC-AUC"],
    "Value": [roc_auc]
})

print("=== Performance summary ===")
display(metrics_rf_weighted)

print("=== Confusion matrix ===")
display(pd.DataFrame(
    confusion_matrix(y_test, y_pred_rf_weighted),
    index=["True 0", "True 1"],
    columns=["Pred 0", "Pred 1"]
))

print("=== Classification report ===")
print(classification_report(y_test, y_pred_rf_weighted, digits=4))

=== Performance summary ===


,Metric,Value
0,ROC-AUC,0.588131


=== Confusion matrix ===


,Pred 0,Pred 1
True 0,3337,1690
True 1,400,338


=== Classification report ===
              precision    recall  f1-score   support

           0     0.8930    0.6638    0.7615      5027
           1     0.1667    0.4580    0.2444       738

    accuracy                         0.6375      5765
   macro avg     0.5298    0.5609    0.5030      5765
weighted avg     0.8000    0.6375    0.6953      5765



Le modèle Random Forest entraîné avec class_weight="balanced" obtient un ROC-AUC de 0.588 ce qui indique une capacité de discrimination moyenne mais supérieure au hasard.

La matrice de confusion montre que le modèle détecte 45.8 % des résiliations (rappel de la classe 1), ce qui traduit une bonne sensibilité à la classe minoritaire. En revanche, cette amélioration du rappel s’accompagne d’un nombre élevé de faux positifs, ce qui se reflète dans une précision faible (16.7 %).

Le F1-score de 0.244 montre un compromis entre détection et fiabilité.

Le modèle Random Forest entraîné sur un jeu d’apprentissage rééquilibré par SMOTE obtient un ROC-AUC de 0.580 ce qui est légèrement inférieur à celui du modèle pondéré.

Le rappel de la classe 1 diminue à 33.3 %, indiquant une capacité plus faible à détecter les résiliations. En contrepartie, le nombre de faux positifs est réduit, ce qui se traduit par une précision légèrement supérieure (17.8 %). Toutefois, cette amélioration reste marginale et ne compense pas la perte de rappel.

Le F1-score de 0.232 confirme que le rééquilibrage par SMOTE n’apporte pas de gain global en termes de performance sur la classe minoritaire.

Dans un contexte de prédiction de résiliation, où l’objectif principal est d’identifier un maximum de clients à risque afin de mettre en place des actions préventives, la pondération des classes apparaît plus pertinente. Elle permet de détecter un plus grand nombre de résiliations potentielles.

En revanche, si l’on souhaite limiter le nombre de faux positifs et obtenir un modèle plus prudent, une approche comme SMOTE peut être envisagée.

L’ensemble des résultats et des analyses présentés dans ce notebook sera synthétisé et approfondie dans le rapport final.